In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)

nDays = 30

days = pd.date_range('2025-01-01', periods=nDays)

# Stable case (slightly different trend)
stable = pd.DataFrame({
    'day': days,
    'accuracy': np.clip(0.88 + 0.0002*np.arange(len(days)) + np.random.normal(0,0.01,len(days)), 0, 1),
    'precision': np.clip(0.86 + 0.00025*np.arange(len(days)) + np.random.normal(0,0.012,len(days)), 0, 1),
    'recall': np.clip(0.87 + 0.00015*np.arange(len(days)) + np.random.normal(0,0.013,len(days)), 0, 1),
})

# Unstable case
unstable = pd.DataFrame({
    'day': days,
    'accuracy': np.clip(np.random.normal(0.90, 0.05, len(days)), 0, 1),
    'precision': np.clip(np.random.normal(0.88, 0.06, len(days)), 0, 1),
    'recall': np.clip(np.random.normal(0.89, 0.07, len(days)), 0, 1),
})

In [3]:
stable.head()

,day,accuracy,precision,recall
0,2025-01-01,0.884967,0.852780,0.863771
1,2025-01-02,0.878817,0.882477,0.867736
2,2025-01-03,0.886877,0.860338,0.855918
3,2025-01-04,0.895830,0.848057,0.854899
4,2025-01-05,0.878458,0.870871,0.881163


In [4]:
unstable.head()

,day,accuracy,precision,recall
0,2025-01-01,0.904854,0.927462,0.907534
1,2025-01-02,0.948432,0.825437,0.914251
2,2025-01-03,0.864897,0.964168,0.842398
3,2025-01-04,0.883617,0.795889,0.906258
4,2025-01-05,0.880395,0.915211,0.910515


In [50]:
import plotly.graph_objects as go

# Plot function
def plot_metrics(df, title):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['day'], y=df['accuracy'],
        mode='lines+markers',
        name='Accuracy'
    ))

    fig.add_trace(go.Scatter(
        x=df['day'], y=df['precision'],
        mode='lines+markers',
        name='Precision'
    ))

    fig.add_trace(go.Scatter(
        x=df['day'], y=df['recall'],
        mode='lines+markers',
        name='Recall'
    ))

    fig.update_layout(
        title=title,
        xaxis_title="Día",
        yaxis_title="Valor",
        yaxis=dict(range=[0.5, 1.0]),
        height=500,
        width=700,
        legend=dict(orientation="h", y=-0.25),
        template='plotly_dark'
    )

    fig.show()

plot_metrics(stable, 'Caso Estable - Métricas del Modelo')
plot_metrics(unstable, 'Caso Inestable - Métricas del Modelo')


In [48]:
# Simulate classification outputs (4 classes)
def simulate_classes(n_days, n_samples, stability='stable'):
    classes = []
    prev = np.random.choice(4, n_samples)

    for day in range(n_days):
        if stability == 'stable':
            # small drift
            drift_prob = 0.05
        else:
            # large drift
            drift_prob = 0.25
        
        mask_change = np.random.rand(n_samples) < drift_prob
        new_class = prev.copy()
        new_class[mask_change] = np.random.choice(4, mask_change.sum())
        classes.append(new_class)
        prev = new_class
    
    return np.array(classes)

stable_classes = simulate_classes(nDays, 500, 'stable')
unstable_classes = simulate_classes(nDays, 500, 'unstable')

# Compute migration metrics: keep, change, new, old
def compute_migrations(class_matrix):
    daily = {'day': days, 'keep': [], 'change': [], 'new': [], 'old': []}

    prev_set = set(class_matrix[0])

    for i in range(1, len(class_matrix)):
        prev = class_matrix[i-1]
        curr = class_matrix[i]
        
        keep = np.sum(prev == curr)
        change = np.sum(prev != curr)

        curr_set = set(curr)
        new = len(curr_set - prev_set)
        old = len(prev_set - curr_set)

        daily['keep'].append(keep)
        daily['change'].append(change)
        daily['new'].append(new)
        daily['old'].append(old)

        prev_set = curr_set

    # Align day length
    for key in ['keep', 'change', 'new', 'old']:
        daily[key] = [np.nan] + daily[key]

    return pd.DataFrame(daily)

stable_mig = compute_migrations(stable_classes)
unstable_mig = compute_migrations(unstable_classes)

In [7]:
stable_mig.head()

,day,keep,change,new,old
0,2025-01-01,NaN,NaN,NaN,NaN
1,2025-01-02,483.0,17.0,0.0,0.0
2,2025-01-03,486.0,14.0,0.0,0.0
3,2025-01-04,484.0,16.0,0.0,0.0
4,2025-01-05,480.0,20.0,0.0,0.0


In [8]:
unstable_mig.head()

,day,keep,change,new,old
0,2025-01-01,NaN,NaN,NaN,NaN
1,2025-01-02,411.0,89.0,0.0,0.0
2,2025-01-03,414.0,86.0,0.0,0.0
3,2025-01-04,397.0,103.0,0.0,0.0
4,2025-01-05,401.0,99.0,0.0,0.0


In [49]:
# Plot stacked bar (%)
def plot_stacked(df, title):
    # Calcular porcentajes
    df_pct = df.copy()
    totals = df[['keep','change','new','old']].sum(axis=1)
    df_pct[['keep','change','new','old']] = df[['keep','change','new','old']].div(totals, axis=0)

    soft_colors = {
        'keep':   '#3F685F',  # verde agua suave
        'change': '#BD924D',  # amarillo ámbar tenue
        'new':    '#5D7FB6',  # azul pastel
        'old':    '#5F4B72',  # lavanda gris suave
    }

    fig = go.Figure()

    for key in ['keep', 'change', 'new', 'old']:
        fig.add_trace(go.Bar(
            x=df_pct['day'],
            y=df_pct[key],
            name=key.capitalize(),
            marker=dict(
                color=soft_colors[key],
                line=dict(width=0)   # ⭐ Sin bordes
            ),
        ))

    fig.update_layout(
        title=title,
        barmode='stack',

        # 🎨 Soft dark theme
        paper_bgcolor="#1D1D1D",
        plot_bgcolor='#1D1D1D',
        font=dict(color='#CDD6F4'),

        xaxis=dict(
            title='Día',
            tickangle=45,
            showgrid=False,
            gridcolor="#313131",
            linecolor='#313131',
        ),
        yaxis=dict(
            title='Porcentaje',
            gridcolor='#313131',
            linecolor='#313131',
        ),

        legend=dict(
            orientation='h',
            y=-0.2,
            font=dict(color='#CDD6F4')
        ),

        height=500,
        width=700,
    )

    fig.show()

plot_stacked(stable_mig, 'Migración Diaria (Stable Case)')
plot_stacked(unstable_mig, 'Migración Diaria (Stable Case 2)')

In [51]:
np.random.seed(42)

days = pd.date_range('2025-01-01', periods=nDays)

# --------------------------
# 1. SIMULACIÓN DEL UNIVERSO
# --------------------------
def simulate_unstable_universe(n_days):
    sizes = []
    for i in range(n_days):
        if np.random.rand() < 0.2:
            sizes.append(np.random.randint(300, 650))  # choque fuerte
        else:
            sizes.append(np.random.randint(450, 550))  # rango normal
    
    return sizes

universe_sizes_stable = np.random.randint(480, 520, len(days))
universe_sizes_unstable = simulate_unstable_universe(len(days))


# -----------------------------------------------
# 2. SIMULACIÓN DE CLASES CON MIGRACIÓN EXTREMA
# -----------------------------------------------
def simulate_classes_with_ids(n_days, sizes, instability=False):
    data = []
    previous_ids = set()

    for i in range(n_days):
        n = sizes[i]

        # IDs del día
        if i == 0:
            ids = np.arange(n)
        else:
            ids = set()
            # keep % of old ids
            if instability:
                # keep_ratio = np.random.choice([0.1,0.2,0.5,0.9], p=[0.3,0.3,0.3,0.1])
                keep_ratio = np.random.choice([
                    np.random.uniform(0.05, 0.3),   # Días muy inestables
                    np.random.uniform(0.7, 0.95)    # Días muy estables
                ])
            else:
                keep_ratio = np.random.normal(0.85, 0.03)
                keep_ratio = np.clip(keep_ratio, 0.75, 0.95)

            n_keep = int(min(len(previous_ids), keep_ratio * n))
            keep_ids = set(np.random.choice(list(previous_ids), n_keep, replace=False)) if n_keep > 0 else set()

            # new ids
            n_new = n - n_keep
            new_ids = set(np.max(list(previous_ids))+1 + np.arange(n_new)) if previous_ids else set(np.arange(n_new))

            ids = list(keep_ids | new_ids)

        # asignación de clases
        if instability:
            # picos extremos
            if np.random.rand() < 0.3:
                classes = np.random.choice([0,1,2,3], len(ids), p=[0.7,0.1,0.1,0.1])
            else:
                classes = np.random.choice(4, len(ids))
        else:
            classes = np.random.choice(4, len(ids), p=[0.25,0.25,0.25,0.25])

        df = pd.DataFrame({
            'id': ids,
            'class': classes,
            'day': days[i]
        })

        data.append(df)
        previous_ids = set(ids)

    return data

stable_daily = simulate_classes_with_ids(len(days), universe_sizes_stable, instability=False)
unstable_daily = simulate_classes_with_ids(len(days), universe_sizes_unstable, instability=True)


# ---------------------------
# 3. CÁLCULO DE MIGRACIONES
# ---------------------------
def compute_migrations(daily_data):
    rows = []
    for i in range(len(daily_data)):
        if i == 0:
            rows.append([daily_data[i]['day'].iloc[0], np.nan, np.nan, np.nan, np.nan])
            continue
        
        prev = daily_data[i-1]
        curr = daily_data[i]

        prev_ids = set(prev['id'])
        curr_ids = set(curr['id'])

        keep_ids = prev_ids & curr_ids
        new_ids = curr_ids - prev_ids
        old_ids = prev_ids - curr_ids

        # cambios de clase
        prev_classes = prev.set_index('id').loc[list(keep_ids)]['class']
        curr_classes = curr.set_index('id').loc[list(keep_ids)]['class']

        keep = np.sum(prev_classes.values == curr_classes.values)
        change = np.sum(prev_classes.values != curr_classes.values)

        rows.append([
            curr['day'].iloc[0],
            keep,
            change,
            len(new_ids),
            len(old_ids)
        ])
    
    return pd.DataFrame(rows, columns=['day','keep','change','new','old'])

stable_mig = compute_migrations(stable_daily)
unstable_mig = compute_migrations(unstable_daily)

fig_unstable = plot_stacked(unstable_mig, 'Migración Diaria - Escenario MUY Inestable')
fig_stable = plot_stacked(stable_mig, 'Migración Diaria - Escenario Estable')
